In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 [==============================] - 40s 2us/step


In [3]:
import os
import pathlib

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of directories: 20
Directory names: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
Number of files in comp.graphics: 1000
Some example filenames: ['37261', '37913', '37914', '37915', '37916']


In [4]:
print(open(data_dir / "comp.graphics" / "38987").read())

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



In [5]:
'''From the privous visualisation we recognize that there are header lines that are leaking the file's category. so we need to
handle them'''
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
'''priting the number of files in this category'''
    print("Processing %s, %d files found" % (dirname, len(fnames)))
'''Removing spaces (\n) and headers ( 10 first lines)'''
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

## Shuffle and split the data into training & validation sets

In [6]:
'''In this step we set and split it into two parts ( Train and test sets ).
The way we will split is :  80:20 ratio (train:validation)
'''
#  Becau
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [7]:
from tensorflow.keras.layers import TextVectorization

# To normalize, split, and map strings to integers we need to add a vectorization layer for texts
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
# Now we will create a tensor flow dataset from the output vectorized
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [8]:
# Printing the top 5 words
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [9]:
# Now we will vectorize a text and visualize the output
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3456, 1682,   15,    2, 5776], dtype=int64)

In [10]:
#Dict mapping words to their indices:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [11]:
# we will try to vectorize the same text and we will obtain the same result
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3456, 1682, 15, 2, 5776]

## Load pre-trained word embeddings

In [23]:
'''The Dataset we downloaded contains text-encoded vectors of various sizes, but we will use the 100d ones only'''
#First we will import the text file
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), ".keras\\datasets\\glove.6B.100d.txt"
)

embeddings_index = {}
# Here we will count number of word vectors, we need to specify the UTF-8 encoder
with open(path_to_glove_file, encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [24]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

''' we will prepare an embedding matrix'''
embedding_matrix = np.zeros((num_tokens, embedding_dim))
'''The entry at index i is the pre-trained vector for the word of index i in our vectorizer's vocabulary'''
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 18019 words (1981 misses)


## Build the model

In [25]:
from tensorflow.keras.layers import Embedding
'''Before building the model we will the embedding layer and set trainable to False to keep the embeddings fixed'''
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [26]:
from tensorflow.keras import layers
'''
Now we will build a simple 1D convnet starting with an Embedding layer.
Before fitting our model, we choosed ReLU as loss function from hiddel layer, and softmax activation function for output layer.
'''

# A integer input for vocab indices.
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)

# Conv1D + global max pooling
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a softmax:
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000200   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         64128     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 128)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 128)        0         
 1D)                                                         

In [27]:
# Fit the model using the train and test datasets.
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [28]:
# because we're doing softmax classification we will use categorical crossentropy as our loss
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 18s 98ms/step - loss: 2.6658 - acc: 0.1392 - val_loss: 2.1176 - val_acc: 0.2598
Epoch 2/20
125/125 [==============================] - 13s 106ms/step - loss: 1.9241 - acc: 0.3368 - val_loss: 1.7784 - val_acc: 0.3936
Epoch 3/20
125/125 [==============================] - 13s 106ms/step - loss: 1.5059 - acc: 0.4826 - val_loss: 1.3214 - val_acc: 0.5371
Epoch 4/20
125/125 [==============================] - 16s 124ms/step - loss: 1.2722 - acc: 0.5628 - val_loss: 1.1670 - val_acc: 0.6004
Epoch 5/20
125/125 [==============================] - 18s 140ms/step - loss: 1.1028 - acc: 0.6239 - val_loss: 1.0947 - val_acc: 0.6304
Epoch 6/20
125/125 [==============================] - 16s 129ms/step - loss: 0.9736 - acc: 0.6660 - val_loss: 1.0277 - val_acc: 0.6492
Epoch 7/20
125/125 [==============================] - 15s 117ms/step - loss: 0.8484 - acc: 0.7015 - val_loss: 1.0208 - val_acc: 0.6592
Epoch 8/20
125/125 [==============================] - 15

In [29]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 1s 577ms/step


'comp.graphics'

In this example we have build and trained a text classification model with the Newsgroup20 dataset. We create a vectorization layer for word splitting & indexing. Because our model is using pre-trained word embeddings we prepared an embedding matrix that we can use in a Keras Embedding layer